In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama3"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

model.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(wait for it...)\n\nBecause it was two-tired!\n\nHope that made you smile!"

In [2]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke")

"Here's one:\n\nWhy couldn't the bicycle stand up by itself?\n\n(Wait for it...)\n\nBecause it was two-tired!\n\nHope that made you laugh! Do you want to hear another one?"

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader

loader = UnstructuredMarkdownLoader("Q&A.md")
pages = loader.load_and_split()
pages

[Document(page_content='1-Getting_Started\n\nGetting Started\n\nQ: What is a Reservoir Computing architecture?\nA: A Reservoir Computing (RC) architecture is a type of recurrent neural network (RNN) where the recurrent layer, called the reservoir, consists of randomly and recurrently connected neurons. This reservoir projects input data into a high-dimensional space to encode temporal information. The only part of the network that is trained is the output layer, called the readout, typically using simple linear regression.\n\nRC is known for its efficient and fast training process. It includes techniques such as Echo State Networks (ESNs) and Liquid State Machines (LSMs). RCs are similar to Support Vector Machines (SVMs) in transforming inputs into dynamic, non-linear, high-dimensional representations. They are used in various tasks, including time series forecasting and sequence generation.\n\nFor more detailed information, you can refer to the document on Reservoir Computing, the res

In [4]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know the answer, and it is because of you".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="Here is a question")

'\nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know the answer, and it is because of you".\n\nContext: Here is some context\n\nQuestion: Here is a question\n'

In [5]:
chain = prompt | model | parser

In [6]:
chain.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}}}

In [7]:
chain.invoke(
    {
        "context": "The name I was given is Brumble",
        "question": "What's my name?"
    }
)

'Based on the context, I would say:\n\nYour name is Brumble!'

In [8]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

In [11]:
retriever = vectorstore.as_retriever()

retriever.invoke("Reservoir")

[Document(page_content='Q: What are the teacher vectors?\nA: Teacher vectors are the target outputs used during the training of a machine learning model. In supervised learning, each input example ( x_i ) in the training dataset has a corresponding teacher vector ( y_i ), which represents the desired output for that input. The model learns by adjusting its parameters to minimize the difference between its predictions and these teacher vectors, thereby enabling it to make accurate predictions on new, unseen data.\n\nDefine a training task\n\nQ: Why do we need to define a training task?\nA: Defining a training task is essential because it specifies the objective the model needs to achieve, such as predicting future values or classifying data. It involves providing input-output pairs (training data) that guide the model in learning the relationship between inputs and desired outputs. This process allows the model to adjust its parameters to minimize errors and generalize well to new data,

In [12]:
from operator import itemgetter

chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | prompt
    | model
    | parser
)

chain.invoke({"question": "What is an ESN? give me more details, what site can I go to learn more about ESN?"})

'ESN stands for Echo State Network. It\'s a type of Reservoir Computing (RC) architecture that combines the echo state property with feedback connections.\n\nReservoir Computing is a family of recurrent neural networks that use a fixed reservoir and only adjust the weights from the reservoir to the readout layer. The echo state property (ESP) is a fundamental concept in RC, which states that the spectral radius of the reservoir should be less than 1 to ensure that the network\'s internal state will not grow indefinitely.\n\nAn ESN is a specific type of RC architecture that uses an artificial neural network as its reservoir. It consists of:\n\n1. A reservoir: This is the main component of the ESN, which is typically implemented using a feedforward neural network with a fixed number of units and nonlinear activation functions.\n2. Feedback connections: The output from the readout layer is fed back into the reservoir, allowing the internal state to evolve over time.\n\nESNs have been show